<a href="https://colab.research.google.com/github/epistimi4/emotion_based_music_recomendation/blob/master/Preprocessing/Spotify_MusicInformationRetrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music Information Retrieval from Spotify #

In [1]:
!pip install spotipy

  Created wheel for spotipy: filename=spotipy-2.4.4-cp36-none-any.whl size=12336 sha256=f6a8617550d44f064af815ecb027b9ef9e1b060d7325140632b2d7be3e0799d5
  Stored in directory: /root/.cache/pip/wheels/76/28/19/a86ca9bb0e32dbd4a4f580870250f5aeef852870578e0427e6
Successfully built spotipy


In [0]:
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #to access authorised Spotify data
import json
import pandas as pd

### Authorization flows ###

#### deprecated using Spotipy library ####

In [0]:
#deprecated
#Temporary user authorization: Implicit Grant
auth_request = requests.get(
    'https://accounts.spotify.com/authorize?client_id=8038c4be0f8a4844982edce1b2282ebc&\
    redirect_uri=http:%2F%2Fexample.com%2Fcallback&scope=user-read-private%20user-read-email&response_type=token&state=123')

print(auth_request.content)
response_dict = json.loads(auth_request.text)

for i in response_dict:
    print("key: ", i, "val: ", response_dict[i])

In [0]:
access_token = '1bd8da3a234e4060a6801a432c0e877d'
artist_name = 'Nibana'

artist_info = requests.get(
    'https://api.spotify.com/v1/search',
    headers={ 'access_token': access_token },
    params={ 'q': artist_name, 'type': 'artist' })

print(artist_info)

<Response [401]>


### Using spotipy ###

#### Authorize app ####

In [0]:
client_id = '8038c4be0f8a4844982edce1b2282ebc'
client_secret = '1bd8da3a234e4060a6801a432c0e877d'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

### Extract metadata /audio features from a song ###

In [0]:
# extract audio features for each song
def audio_features(song_name, track_id):    
    spotify_song = {}
  
    #for track in songs_found:
        #pull audio features per track
        
    features = sp.audio_features(track_id)
    spotify_song['id'] = features[0]['id']
    #Append to relevant key-value
    spotify_song['acousticness'] = features[0]['acousticness']
    spotify_song['danceability'] = features[0]['danceability']
    spotify_song['energy'] = features[0]['energy']
    spotify_song['instrumentalness'] = features[0]['instrumentalness']
    spotify_song['liveness'] = features[0]['liveness']
    spotify_song['loudness'] = features[0]['loudness']
    spotify_song['speechiness'] = features[0]['speechiness']
    spotify_song['tempo'] = features[0]['tempo']
    spotify_song['valence'] = features[0]['valence']
    #popularity is stored elsewhere
    pop = sp.track(track_id)#track['id'])
    spotify_song['popularity'] = pop['popularity']
    spotify_song['song_name'] = pop['name']
    
    #if spotify_song['song_name'].lower() == song_name.lower():
    print(song_name, spotify_song['song_name'])
    #if not any(d['song_name'] == spotify_song['song_name'] for d in spotify_songs):
    #  spotify_songs.append(spotify_song)          

    return spotify_song

### Search spotify for the songs in my list ###

In [53]:
songs_found = []
song_list = ["our eyes (sephira remix)", "the grid", 'Sirius 2017(Disco Demolition Remix)', 'Project L.O.V.E', 'Tron Legacy - Son Of Flynn', 'Blue... The Color of Dreams']
print('my playlist has {0} songs'.format(len(song_list)))

for name in song_list:  #chosen track
  result = sp.search(name) #search query
  track_id = result['tracks']['items'][0]['id']
  #print(track_id, result['tracks']['items'][0])
  #print(sp.audio_features([track_id]))
  songs_found.append(audio_features(name, track_id))#result['tracks']['items']) )

print('added {0} tracks'.format(len(songs_found)))

my playlist has 6 songs
our eyes (sephira remix) Our Eyes - Sephira Remix
the grid Off The Grid
Sirius 2017(Disco Demolition Remix) Sirius 2017 - Disco Demolition Remix
Project L.O.V.E Project L.O.V.E
Tron Legacy - Son Of Flynn The Son of Flynn
Blue... The Color of Dreams Blue... The Color of Dreams
added 6 tracks


In [56]:
df = pd.io.json.json_normalize(songs_found)
#display(list(songs_found))
df.head(10)

,id,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,song_name
0,3yzlDHdP9buTr9jkZfUyEq,0.055400,0.642,0.833,0.865000,0.272,-8.621,0.0358,100.015,0.0493,21,Our Eyes - Sephira Remix
1,5STtg0bZXSoWfF20qWGxeV,0.054900,0.734,0.899,0.000039,0.108,-4.442,0.0669,124.035,0.5200,59,Off The Grid
2,5ld9arEKsxvJr7RqoWd337,0.077700,0.533,0.864,0.904000,0.130,-7.941,0.0394,119.023,0.0787,40,Sirius 2017 - Disco Demolition Remix
3,0gX7U91bqxtZVLIZ0MhsTC,0.000242,0.575,0.908,0.537000,0.440,-7.535,0.0345,126.001,0.0713,37,Project L.O.V.E
4,1xPec5BN0Zxv77zrWKq43S,0.901000,0.543,0.775,0.902000,0.258,-17.616,0.0520,106.018,0.0222,55,The Son of Flynn
5,2EcJA9RZSfsLSq6cQOLtMu,0.393000,0.521,0.660,0.831000,0.117,-10.207,0.0260,119.996,0.1610,18,Blue... The Color of Dreams
